In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import seaborn as sns
import joblib # For saving the model

# 1. Load the Vetoed Dataset
# This has features (RSI, etc.) and Safety Labels (0 for NFP/Risky days)
df = pd.read_parquet("../data/EURUSD_D1_Ready_for_XGBoost_Vetoed.parquet")

print(f"Loaded {len(df)} rows.")

# 2. Prepare X (Features) and y (Target)
# We drop columns that the model can't see or shouldn't use (like 'open', 'high', 'close')
# We only want the calculated indicators.
features_to_drop = ['open', 'high', 'low', 'close', 'tick_volume', 'spread', 'atr', 'label', 'real_volume']
# Note: We keep 'label' as y, but drop it from X.

X = df.drop(columns=features_to_drop, errors='ignore')
y = df['label']

# Remap labels for XGBoost
# XGBoost expects classes to be [0, 1, 2].
# Our labels are [-1, 0, 1]. Let's map them:
# -1 (Sell) -> 0
#  0 (Wait) -> 1
#  1 (Buy)  -> 2
label_mapping = {-1: 0, 0: 1, 1: 2}
y_mapped = y.map(label_mapping)

print(f"Features used ({len(X.columns)}): {X.columns.tolist()}")

# 3. Time Series Split (The "No Cheating" Split)
# We split by TIME, not randomly.
# Train: First 80% of data (Past)
# Test: Last 20% of data (Future relative to train)
split_point = int(len(df) * 0.80)

X_train = X.iloc[:split_point]
y_train = y_mapped.iloc[:split_point]

X_test = X.iloc[split_point:]
y_test = y_mapped.iloc[split_point:]

print(f"\nTraining on {len(X_train)} days (Past).")
print(f"Testing on {len(X_test)} days (Future).")

# 4. Initialize and Train XGBoost
# We use 'multi:softprob' because we have 3 classes (Sell, Wait, Buy)
model = xgb.XGBClassifier(
    n_estimators=100,      # Number of "trees" (decisions)
    learning_rate=0.05,    # Speed of learning (Lower = slower but more precise)
    max_depth=4,           # Complexity of trees (3-5 is good to prevent overfitting)
    objective='multi:softprob', 
    num_class=3,
    eval_metric='mlogloss',
    subsample=0.8,         # Use 80% of data per tree (adds randomness/robustness)
    colsample_bytree=0.8,  # Use 80% of features per tree
    random_state=42,
    n_jobs=-1              # Use all CPU cores
)

print("\n🧠 Training Supervisor Model...")
model.fit(X_train, y_train)
print("✅ Training Complete.")

# 5. Make Predictions
y_pred = model.predict(X_test)

# 6. Evaluation
# We need to map predictions back to our human-readable labels (-1, 0, 1)
# Inverse map: 0->-1, 1->0, 2->1
inverse_map = {0: -1, 1: 0, 2: 1}
y_test_human = y_test.map(inverse_map)
y_pred_human = pd.Series(y_pred).map(inverse_map)

print("\n--- CLASSIFICATION REPORT ---")
# This tells us: "When the model said BUY, how often was it right?" (Precision)
print(classification_report(y_test_human, y_pred_human))

# 7. Confusion Matrix Visualization
cm = confusion_matrix(y_test_human, y_pred_human, labels=[-1, 0, 1])
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Sell', 'Wait', 'Buy'], yticklabels=['Sell', 'Wait', 'Buy'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Supervisor Confusion Matrix')
plt.show()

# 8. Feature Importance (What did the model learn?)
xgb.plot_importance(model, importance_type='weight', max_num_features=10)
plt.title("Top 10 Features Used by Supervisor")
plt.show()

# 9. Save the Brain
# We save it so we can load it later for the Backtest and Live Bot
model_filename = "../models/supervisor_xgb.joblib"
joblib.dump(model, model_filename)
print(f"💾 Model saved to {model_filename}")